<a href="https://colab.research.google.com/github/ajay-richardson/Sentiment-Analysis---Major-Project/blob/main/Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#import dataset containing tweets to an airline
raw_data = pd.read_csv("https://raw.githubusercontent.com/ajay-richardson/Sentiment-Analysis---Major-Project/main/Tweets.csv")

#create required dataframe from raw data
list = ['airline_sentiment','text']
df = raw_data[list]
df.rename(columns = {'airline_sentiment':'label','text':'tweet'},inplace = True)

import string
#convert all text to lowercase
df.tweet = df.tweet.apply(lambda x:x.lower())

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB

punctuations = [string.punctuation]
def remove_punctuations(text):
  for punctuation in string.punctuation:
      text = text.replace(punctuation,'')
  return text    

#removing punctuation in text 
df.tweet = df.tweet.apply(remove_punctuations)

nltk.download('stopwords')
stop = stopwords.words('english')
#removing stopwords
df.tweet = df.tweet.apply(lambda x: " ".join(x for x in x.split() if x not in stop))

nltk.download('punkt')
#create new column with tokenized sentences
df['tweet_tokenized'] = df.tweet.apply(lambda x: word_tokenize(x))

nltk.download('wordnet')
#lemmatization of each individual sentence
lemmatizer = WordNetLemmatizer()
df['tweet_lemmatized'] = df.tweet_tokenized.apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

df['tweet_lemmatized_train'] = df.tweet_lemmatized.apply(lambda x: ' '.join(x))
X = df['tweet_lemmatized_train']
y = df['label']

le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

vect = TfidfVectorizer(smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word')
bow_fit = vect.fit(X)
bow = bow_fit.transform(X)

from sklearn.model_selection import train_test_split,GridSearchCV, validation_curve
X_train,X_test,y_train,y_test = train_test_split(bow,y,random_state=42)

mnb = MultinomialNB(alpha = 1.75)
mnb.fit(X_train,y_train)
y_pred = mnb.predict(X_test)
#Labels for '['negative' 'neutral' 'positive']' are '[0 1 2]' respectively.

import streamlit as st
from sklearn.pipeline import Pipeline

st.title("Message Sentiment Classifier")
st.subheader('TFIFD Vectorizer')
st.write('This project is based on Naive Bayes Classifier')

text_model = Pipeline([('tfidf',TfidfVectorizer()),('model',MultinomialNB(alpha = 1.75))])
text_model.fit(X,y)
message = st.text_area("Enter Text","Type Here ..")
op = text_model.predict([message])
if st.button("Predict"):
  st.title(op)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [2]:
!pip install streamlit

In [3]:
!pip install pyngrok==4.1.1

In [4]:
from pyngrok import ngrok
url = ngrok.connect(port='8501')
url

2020-11-17 13:53:00.305 INFO    pyngrok.process: ngrok process starting: 999
2020-11-17 13:53:00.316 INFO    pyngrok.process: t=2020-11-17T13:53:00+0000 lvl=info msg="no configuration paths supplied"

2020-11-17 13:53:00.321 INFO    pyngrok.process: t=2020-11-17T13:53:00+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

2020-11-17 13:53:00.325 INFO    pyngrok.process: t=2020-11-17T13:53:00+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil

2020-11-17 13:53:00.327 INFO    pyngrok.process: t=2020-11-17T13:53:00+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040

2020-11-17 13:53:01.081 INFO    pyngrok.process: t=2020-11-17T13:53:01+0000 lvl=info msg="tunnel session started" obj=tunnels.session

2020-11-17 13:53:01.082 INFO    pyngrok.process: t=2020-11-17T13:53:01+0000 lvl=info msg="client session established" obj=csess id=b108026df11a

2020-11-17 13:53:01.087 INFO    pyngrok.process: ngrok process has s

'http://037c3e55b1ec.ngrok.io'


2020-11-17 13:53:01.476 INFO    pyngrok.process: t=2020-11-17T13:53:01+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8501-5b753037-4be0-43b7-a36e-5d2f80b79077 addr=http://localhost:8501 url=https://037c3e55b1ec.ngrok.io



In [ ]:
!streamlit run major_project.py &

2020-11-17 13:53:01.498 INFO    pyngrok.process: t=2020-11-17T13:53:01+0000 lvl=info msg=end pg=/api/tunnels id=0f83e6357e94e228 status=201 dur=373.26007ms



/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.229.160.200:8501

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

2020-11-17 13:53:15.042 INFO    pyngrok.process: t=2020-11-17T13:53:15+0000 lvl=info msg="join connections" obj=join id=236aede984ea l=127.0.0.1:8501 r=122.171.231.129:59922

2020-11-17 13:53:15.682 INFO    pyngrok.process: t=2020-11-17T13:53:15+0000 lvl=info msg="join connections" obj=join id=5258090b24f5 l=127.0.0.1:8501 r=122.171.231.129:17327

2020-11-17 13:53:15.703 INFO    pyngrok.process: t=2020-11-17T13:53:15+0000 lvl=info msg="join connections" obj=join id=3959c9d40123 l=127.0.0.1:8501 r=122.171.231.129:59922

2020-11-17 13:53:20.308 INFO    pyngrok.process: t=2020-11-17T13:53:20+0000 lvl=info msg="join connections" obj=join id=411a4b5c00df l=127.0.0.1:8501 r=122.171.231.129:59929

2020-11-17 13:53:21.857 INFO    pyngrok.process: t=2020-11-17T13:53:21+0000 lvl=info msg="join connections" obj=join id=8204a2ece8b1 l=127.0.0.1:8501 r=122.171.231.129:17333



/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/content/major_project.py:43: SettingWithCopyWarning: 
A value is trying to be 

2020-11-17 13:53:27.022 INFO    pyngrok.process: t=2020-11-17T13:53:27+0000 lvl=info msg="join connections" obj=join id=18cf26119c93 l=127.0.0.1:8501 r=122.171.231.129:59924

2020-11-17 13:53:27.029 INFO    pyngrok.process: t=2020-11-17T13:53:27+0000 lvl=info msg="join connections" obj=join id=d769c8f6630a l=127.0.0.1:8501 r=122.171.231.129:59922

2020-11-17 13:53:48.864 INFO    pyngrok.process: t=2020-11-17T13:53:48+0000 lvl=info msg="join connections" obj=join id=2db07249a38b l=127.0.0.1:8501 r=122.171.231.129:17341



/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/content/major_project.py:43: SettingWithCopyWarning: 
A value is trying to be 